## Introduction
This notebook is intented for the users that experience difficulties executing the voila dashboard contained in UI.ipynb. It provides the same fonctionalities that you can run and use by clicking on : 
```
Kernel -> Restart & Run All
```

In [1]:
%%capture
import sys
!{sys.executable} -m pip install 'sepal-ui>=0.7.1-beta'

In [2]:
%run 'table_ui.ipynb'
%run 'viz_ui.ipynb'
%run 'export_ui.ipynb'
%run 'about_ui.ipynb'

In [3]:
file_tile

Tile(align_center=True, children=[Card(children=[Html(children=['Select File'], tag='h2'), Flex(children=[Sele…

In [4]:
col_tile

Tile(align_center=True, children=[Card(children=[Html(children=['Select Columns'], tag='h2'), Flex(children=[S…

In [5]:
tb_map_tile

Tile(align_center=True, children=[Card(children=[Html(children=['Display points'], tag='h2'), Flex(children=[S…

In [6]:
viz_input

Tile(align_center=True, children=[Card(children=[Html(children=['select your vizualisation Parameters'], tag='…

In [7]:
export_tile

Tile(align_center=True, children=[Card(children=[Html(children=['Export data'], tag='h2'), Flex(children=[''],…

In [8]:
export_results

Tile(align_center=True, children=[Card(children=[Html(children=['Export results'], tag='h2'), Flex(children=['…

In [9]:
ts_about

TileAbout(align_center=True, children=[Card(children=[Html(children=['About'], tag='h2'), Flex(children=[MyHTM…

In [10]:
ts_disclaimer

TileDisclaimer(align_center=True, children=[Card(children=[Html(children=['Disclaimer'], tag='h2'), Flex(child…